# **模型調校（Model Tuning）**
此份程式碼會提供針對某資料集的模型調校策略，以及比較其超參數的選擇。

## 本章節內容大綱
* ### [損失函數（Loss function）](#LossFunction)
* ### [激活函數（Activation function）](#ActivationFunction)
* ### [優化器（Optimizer）](#Optimizer)
* ### [學習率（Learning rate）](#LearningRate)
* ### [模型架構（Model architecture）](#ModelArchitecture)

## 匯入套件

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Tensorflow 相關套件
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## 創建資料集／載入資料集（Dataset Creating / Loading）

In [ ]:
# 上傳資料
!wget -q https://github.com/TA-aiacademy/course_3.0/releases/download/DL/Data_part3.zip
!unzip -q Data_part3.zip

In [ ]:
train_df = pd.read_csv('./Data/News_train.csv')
test_df = pd.read_csv('./Data/News_test.csv')

In [ ]:
train_df.head()

* #### 新聞文章資料集
訓練集，測試集分別為 7728，1907 筆，4081 種常用字詞，若在同一篇新聞中出現該字詞為 1，若否則為 0，y_category 標記文章類別，共 11 種類別。

In [ ]:
X_df = train_df.iloc[:, :-1].values
y_df = train_df.y_category.values

In [ ]:
X_test = test_df.iloc[:, :-1].values
y_test = test_df.y_category.values

## 資料前處理（Data Preprocessing）

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Feature scaling
sc = StandardScaler()
X_scale = sc.fit_transform(X_df, y_df)
X_test_scale = sc.transform(X_test)

In [ ]:
# Convert to One-Hot encoding
y_onehot = keras.utils.to_categorical(y_df)
y_test_onehot = keras.utils.to_categorical(y_test)

In [ ]:
# train, valid/test dataset split
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_scale, y_onehot,
                                                      test_size=0.2,
                                                      random_state=17,
                                                      stratify=y_df)

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'X_valid shape: {X_valid.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_valid shape: {y_valid.shape}')

## 模型建置（Model Building）

In [ ]:
def build_model(input_shape, output_shape):
    keras.backend.clear_session()  # 重置 keras 的所有狀態
    tf.random.set_seed(17)  # 設定 tensorflow 隨機種子

    model = keras.models.Sequential()
    model.add(layers.Dense(16,  # 神經元個數
                           input_shape=input_shape,  # 輸入形狀
                           activation='sigmoid'))  # 激活函數
    model.add(layers.Dense(16,
                           activation='sigmoid'))
    model.add(layers.Dense(output_shape,
                           activation='softmax'))
    return model

In [ ]:
model = build_model(X_train[0].shape, y_onehot.shape[1])
model.summary()

## 模型訓練（Model Training）

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=64,
                    validation_data=(X_valid, y_valid))

## 模型評估（Model Evaluation）

In [ ]:
train_loss = history.history['loss']
train_acc = history.history['acc']

valid_loss = history.history['val_loss']
valid_acc = history.history['val_acc']

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1)
plt.plot(range(len(train_loss)), train_loss, label='train_loss')
plt.plot(range(len(valid_loss)), valid_loss, label='valid_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(len(train_acc)), train_acc, label='train_acc')
plt.plot(range(len(valid_acc)), valid_acc, label='valid_acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## 模型調校

![](https://i.imgur.com/WY7awUI.png)

<a name="LossFuction"></a>
* ## 損失函數（Loss function）
tf.keras.losses: https://www.tensorflow.org/api_docs/python/tf/keras/losses

In [ ]:
# 以下放置要比較的 loss function
loss_funcs = ['mean_squared_error',
              'categorical_crossentropy',
              'mean_absolute_error']

batch_size = 64
epochs = 20

# 建立兩個 list 記錄選用不同 loss function 的訓練結果
all_loss, all_acc = [], []

# 迭代不同的 loss function 去訓練模型
for loss_f in loss_funcs:
    print(f'Running model, loss = {loss_f}')

    # 確保每次都是訓練新的模型，而不是接續上一輪的模型
    model = build_model(X_train[0].shape, y_onehot.shape[1])
    model.compile(loss=loss_f,
                  optimizer='rmsprop',
                  metrics=['acc'])

    # 確保每次都設定一樣的參數
    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0,
                        validation_data=(X_valid, y_valid))

    # 將訓練過程記錄下來
    all_loss.append(history.history['loss'])
    all_acc.append(history.history['acc'])
print('----------------- training done! -----------------')

In [ ]:
# 視覺化訓練過程
plt.figure(figsize=(15, 7))

# 繪製 Training loss
plt.subplot(121)
for k in range(len(loss_funcs)):
    plt.plot(range(len(all_loss[k])), all_loss[k], label=loss_funcs[k])
plt.title('Loss')

# 繪製 Training accuracy
plt.subplot(122)
for k in range(len(loss_funcs)):
    plt.plot(range(len(all_acc[k])), all_acc[k], label=loss_funcs[k])
plt.title('Accuracy')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=2.)
plt.ylim((0, 1))
plt.show()

---
![](https://i.imgur.com/vBnVoRI.png)

---

<a name="ActivationFuction"></a>
* ## 激活函數（Activation function）
tf.keras.activations: https://www.tensorflow.org/api_docs/python/tf/keras/activations

In [ ]:
def build_model_activation(input_shape, output_shape, activation):
    # 重新建構一個可以更改 Activation 的模型
    keras.backend.clear_session()
    tf.random.set_seed(17)

    model = keras.models.Sequential()
    model.add(layers.Dense(16,
                           input_shape=input_shape))
    if activation == 'leakyrelu':
        model.add(layers.LeakyReLU())  # LeakyReLU 位置於 Layers 底下
    else:
        model.add(layers.Activation(activation))  # 由此更改 Activation

    model.add(layers.Dense(16))
    if activation == 'leakyrelu':
        model.add(layers.LeakyReLU())  # LeakyReLU 位置於 Layers 底下
    else:
        model.add(layers.Activation(activation))  # 由此更改 Activation

    model.add(layers.Dense(output_shape,
                           activation='softmax'))
    return model

In [ ]:
# 以下放置要比較的 activation function
activation_funcs = ['linear',
                    'sigmoid',
                    'tanh',
                    'relu',
                    'softplus',
                    'leakyrelu',
                    'swish']

batch_size = 64
epochs = 20

# 建立兩個 list 記錄選用不同 activation function 的訓練結果
all_loss, all_acc = [], []

# 迭代不同的 activation function 去訓練模型
for activation_f in activation_funcs:
    print(f'Running model, activation = {activation_f}')

    # 確保每次都是訓練新的模型，而不是接續上一輪的模型
    model = build_model_activation(X_train[0].shape,
                                   y_onehot.shape[1],
                                   activation_f)
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])

    # 確保每次都設定一樣的參數
    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0,
                        validation_data=(X_valid, y_valid))

    # 將訓練過程記錄下來
    all_loss.append(history.history['loss'])
    all_acc.append(history.history['acc'])
print('----------------- training done! -----------------')

In [ ]:
# 視覺化訓練過程
plt.figure(figsize=(15, 7))

# 繪製 Training loss
plt.subplot(121)
for k in range(len(activation_funcs)):
    plt.plot(range(len(all_loss[k])), all_loss[k], label=activation_funcs[k])
plt.title('Loss')

# 繪製 Training accuracy
plt.subplot(122)
for k in range(len(activation_funcs)):
    plt.plot(range(len(all_acc[k])), all_acc[k], label=activation_funcs[k])
plt.title('Accuracy')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.ylim((0, 1))
plt.show()

In [ ]:
# 視覺化訓練過程
plt.figure(figsize=(15, 4))

# 繪製 Training loss
plt.subplot(121)
for k in range(len(activation_funcs)):
    plt.plot(range(len(all_loss[k])), all_loss[k], label=activation_funcs[k])
plt.title('Loss')
plt.ylim((0, 0.3))

# 繪製 Training accuracy
plt.subplot(122)
for k in range(len(activation_funcs)):
    plt.plot(range(len(all_acc[k])), all_acc[k], label=activation_funcs[k])
plt.title('Accuracy')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.ylim((0.95, 0.975))
plt.show()

---
![](https://i.imgur.com/r49pQ2C.png)

---

<a name="Optimizer"></a>
* ## 優化器（Optimizer）
tf.keras.optimizers: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers

In [ ]:
# 以下放置要比較的 optimizer
optimizer_funcs = ['sgd',
                   'rmsprop',
                   'adam',
                   'nadam']

batch_size = 64
epochs = 20

# 建立兩個 list 記錄選用不同 optimizer 的訓練結果
all_loss, all_acc = [], []

# 迭代不同的 optimizer 去訓練模型
for optimizer_f in optimizer_funcs:
    print(f'Running model, optimizer = {optimizer_f}')

    # 確保每次都是訓練新的模型，而不是接續上一輪的模型
    model = build_model_activation(X_train[0].shape,
                                   y_onehot.shape[1],
                                   'tanh')
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer_f,
                  metrics=['acc'])

    # 確保每次都設定一樣的參數
    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0,
                        validation_data=(X_valid, y_valid))

    # 將訓練過程記錄下來
    all_loss.append(history.history['loss'])
    all_acc.append(history.history['acc'])
print('----------------- training done! -----------------')

In [ ]:
# 視覺化訓練過程
plt.figure(figsize=(15, 7))

# 繪製 Training loss
plt.subplot(121)
for k in range(len(optimizer_funcs)):
    plt.plot(range(len(all_loss[k])), all_loss[k], label=optimizer_funcs[k])
plt.title('Loss')

# 繪製 Training accuracy
plt.subplot(122)
for k in range(len(optimizer_funcs)):
    plt.plot(range(len(all_acc[k])), all_acc[k], label=optimizer_funcs[k])
plt.title('Accuracy')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=2.)
plt.ylim((0, 1))
plt.show()

<a name="LearningRate"></a>
* ## 學習率（Learning rate）

In [ ]:
# 以下放置要比較的 learning rate
lr_list = [0.1, 0.01, 0.001, 0.0001, 0.00001]

batch_size = 64
epochs = 20

# 建立兩個 list 記錄選用不同 learning rate 的訓練結果
all_loss, all_acc = [], []

# 迭代不同的 learning rate 去訓練模型
for lr in lr_list:
    print(f'Running model, learning rate = {lr}')

    # 確保每次都是訓練新的模型，而不是接續上一輪的模型
    model = build_model_activation(X_train[0].shape,
                                   y_onehot.shape[1],
                                   'tanh')
    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Nadam(lr),
                  metrics=['acc'])

    # 確保每次都設定一樣的參數
    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0,
                        validation_data=(X_valid, y_valid))

    # 將訓練過程記錄下來
    all_loss.append(history.history['loss'])
    all_acc.append(history.history['acc'])
print('----------------- training done! -----------------')

In [ ]:
# 視覺化訓練過程
plt.figure(figsize=(15, 7))

# 繪製 Training loss
plt.subplot(121)
for k in range(len(lr_list)):
    plt.plot(range(len(all_loss[k])), all_loss[k], label=lr_list[k])
plt.title('Loss')

# 繪製 Training accuracy
plt.subplot(122)
for k in range(len(lr_list)):
    plt.plot(range(len(all_acc[k])), all_acc[k], label=lr_list[k])
plt.title('Accuracy')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=2.)
plt.ylim((0, 1))
plt.show()

<a name="ModelArchitecture"></a>
* ## 模型架構（Model architecture）

In [ ]:
def build_model_architecture(input_shape, output_shape, layer, neuron):
    keras.backend.clear_session()
    tf.random.set_seed(17)

    model = keras.models.Sequential()
    for i in range(layer):
        model.add(layers.Dense(neuron,
                               input_shape=input_shape,
                               activation='tanh'))
    model.add(layers.Dense(output_shape,
                           activation='softmax'))
    return model

In [ ]:
# 以下放置要比較的 layers/ neurons
layers_num = [1, 2, 3]
neurons_num = [16, 32, 64]

batch_size = 64
epochs = 20

# 建立兩個 list 記錄選用不同 layers/ neurons 的訓練結果
all_loss, all_acc = [], []

# 迭代不同的 layers/ neurons 去訓練模型
for layer in layers_num:
    for neuron in neurons_num:
        print(f'Running model, (layer, neuron) = {(layer, neuron)}')

        # 確保每次都是訓練新的模型，而不是接續上一輪的模型
        model = build_model_architecture(X_train[0].shape,
                                         y_onehot.shape[1],
                                         layer,
                                         neuron)
        model.compile(loss='categorical_crossentropy',
                      optimizer='nadam',
                      metrics=['acc'])

        # 確保每次都設定一樣的參數
        history = model.fit(X_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=0,
                            validation_data=(X_valid, y_valid))

        # 將訓練過程記錄下來
        all_loss.append(history.history['loss'])
        all_acc.append(history.history['acc'])
print('----------------- training done! -----------------')

In [ ]:
layer_neuron = list(zip(sum([[i]*3 for i in layers_num], []), neurons_num*3))

# 視覺化訓練過程
plt.figure(figsize=(15, 7))

# 繪製 Training loss
plt.subplot(121)
for k in range(len(layer_neuron)):
    plt.plot(range(len(all_loss[k])), all_loss[k], label=layer_neuron[k])
plt.title('Loss')

# 繪製 Training accuracy
plt.subplot(122)
for k in range(len(layer_neuron)):
    plt.plot(range(len(all_acc[k])), all_acc[k], label=layer_neuron[k])
plt.title('Accuracy')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.ylim((0, 1))
plt.show()

In [ ]:
# 視覺化訓練過程
plt.figure(figsize=(15, 4))

# 繪製 Training loss
plt.subplot(121)
for k in range(len(layer_neuron)):
    plt.plot(range(len(all_loss[k])), all_loss[k], label=layer_neuron[k])
plt.title('Loss')
plt.ylim((0.05, 0.1))

# 繪製 Training accuracy
plt.subplot(122)
for k in range(len(layer_neuron)):
    plt.plot(range(len(all_acc[k])), all_acc[k], label=layer_neuron[k])
plt.title('Accuracy')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.ylim((0.96, 0.9725))
plt.show()

---
### Quiz
請試著利用 Data/pkgo_train.csv 做多元分類問題，預測五個種類的 pokemon，並調整模型（網路層數、神經元數目、激活函數）以及訓練相關的參數得到更高的準確度。